In [102]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque

In [103]:
# Clase vértice - Usuários
class User:
    def __init__(self, user: str, email: str):
        """
        user: Usuário da rede
        email: email do usuário
        """
        self.user = user
        self.email = email

    def __str__(self):  # Vai printar o objeto usuário como string
        return str(self.user)

    def __repr__(self): # Representação para depuração
        return f"User(user='{self.user}')"

    def __hash__(self): # Necessário para usar objetos User como chaves em dicionários/sets
        return hash(self.user)

    def __eq__(self, other): # Necessário para comparar objetos User
        return isinstance(other, User) and self.user == other.user

In [104]:
# Clase aresta - Conexões
class Conection:
    def __init__(self, user_1: User, user_2: User, n_interacoes: int = 1):
        self.user_1 = user_1  # Quem segue
        self.user_2 = user_2  # Quem é seguido
        self.n_interacoes = n_interacoes # Peso da aresta

    def __str__(self):
        return f"{self.user_1.user} segue {self.user_2.user}. {self.user_1.user} interagiu {self.n_interacoes} vezes com o perfil de {self.user_2.user}."

In [105]:
# Clase Postagem
class Post:
    next_id = 0 # Para gerar IDs únicos para posts

    def __init__(self, author: User, content: str):
        self.post_id = Post.next_id
        Post.next_id += 1
        self.author = author
        self.content = content
        self.date = datetime.now()
        self.likes = 0

    def __str__(self):
        return f"Post #{self.post_id} de {self.author.user}: {self.content}. {self.date.strftime('%Y-%m-%d %H:%M:%S')}"

In [106]:
# Clase Grafo - Rede Social
class UniPoli:
    def __init__(self):
        self.users = {}  # Mapeia nome do usuário (str) para objeto User
        self.posts = []  # Lista de objetos Post
        self.conections = []  # Lista de objetos Conection (representação de lista de arestas)

        # Lista de adjacência para acesso eficiente aos vizinhos
        # self.adj_list = {User_obj: [User_obj_seguido_1, User_obj_seguido_2, ...]}
        self.adj_list = {}
        
        # AdiciLista de adjacência reversa para acesso eficiente aos seguidores
        # self.rev_adj_list = {User_obj: [User_obj_seguidor_1, User_obj_seguidor_2, ...]}
        self.rev_adj_list = {}

    def addUser(self, user: str, email: str):
        if user in self.users:
            print(f"ERRO ADDUSER: Usuário '{user}' já existe.")
            return None
        new_user = User(user, email)
        self.users[user] = new_user
        
        # Inicializa entradas para o novo usuário nas listas de adjacência
        self.adj_list[new_user] = []
        self.rev_adj_list[new_user] = []
        
        print(f"Usuário '{user}' adicionado com sucesso.")
        return new_user

    def addConection(self, user_1: str, user_2: str, n_interacoes: int = 1):
        if user_1 not in self.users or user_2 not in self.users:
            print(f"ERRO ADDCONECTION: Um ou ambos os usuários não existem.")
            return False

        u1_obj = self.users[user_1]
        u2_obj = self.users[user_2]

        # Verifica se a conexão já existe na lista de arestas
        for c in self.conections:
            if c.user_1 == u1_obj and c.user_2 == u2_obj:
                print(f"Conexão entre '{user_1}' e '{user_2} já existe.")
                return True

        # Cria nova conexão e adiciona à lista de arestas
        new_connection = Conection(u1_obj, u2_obj, n_interacoes)
        self.conections.append(new_connection)

        # Atualiza as listas de adjacência
        self.adj_list[u1_obj].append(u2_obj)
        self.rev_adj_list[u2_obj].append(u1_obj)

        print(f"Conexão adicionada: '{user_1}' segue '{user_2}'.")
        return True

    def delConection(self, user_1: str, user_2: str):
        if user_1 not in self.users or user_2 not in self.users:
            print(f"ERRO DELCONECTION: Um ou ambos os usuários não existem.")
            return False

        u1_obj = self.users[user_1]
        u2_obj = self.users[user_2]

        found_connection = None
        for c in self.conections:
            if c.user_1 == u1_obj and c.user_2 == u2_obj:
                found_connection = c
                break

        if found_connection:
            self.conections.remove(found_connection)
            
            # Remove da lista de adjacência
            if u2_obj in self.adj_list.get(u1_obj, []):
                self.adj_list[u1_obj].remove(u2_obj)
                
            # Remove da lista de adjacência reversa
            if u1_obj in self.rev_adj_list.get(u2_obj, []):
                self.rev_adj_list[u2_obj].remove(u1_obj)
                
            print(f"Conexão entre '{user_1}' e '{user_2}' removida.")
            return True
        else:
            print(f"ERRO DELCONECTION: Conexão entre '{user_1}' e '{user_2}' não encontrada.")
            return False

    def delUser(self, user_name: str):
        if user_name not in self.users:
            print(f"ERRO DELUSER: Usuário '{user_name}' não encontrado.")
            return False

        user_obj_to_delete = self.users[user_name]

        # Remove o usuário do dicionário principal de usuários
        del self.users[user_name]

        # Remove postagens do usuário
        self.posts = [post for post in self.posts if post.author != user_obj_to_delete]

        # Remove todas as conexões (arestas) que envolvem este usuário
        self.conections = [
            c for c in self.conections
            if c.user_1 != user_obj_to_delete and c.user_2 != user_obj_to_delete
        ]

        # Remove o usuário das listas de adjacência e adjacência reversa
        # 1. Remove as entradas do próprio usuário como chave
        if user_obj_to_delete in self.adj_list:
            del self.adj_list[user_obj_to_delete]
            
        if user_obj_to_delete in self.rev_adj_list:
            del self.rev_adj_list[user_obj_to_delete]

        # 2. Remove o usuário de outras listas de adjacência (onde ele era um seguidor ou seguido)
        for u_obj in list(self.adj_list.keys()): # Itera sobre uma cópia das chaves para evitar erro de runtime
            if user_obj_to_delete in self.adj_list.get(u_obj, []):
                self.adj_list[u_obj].remove(user_obj_to_delete)
        for u_obj in list(self.rev_adj_list.keys()): # Itera sobre uma cópia das chaves
            if user_obj_to_delete in self.rev_adj_list.get(u_obj, []):
                self.rev_adj_list[u_obj].remove(user_obj_to_delete)

        print(f"Usuário '{user_name}' e suas interações removidas com sucesso.")
        return True

    def create_post(self, author_name: str, content: str):
        if author_name not in self.users:
            print(f"ERRO CREATE_POST: Autor '{author_name}' não encontrado.")
            return None
        author_obj = self.users[author_name]
        new_post = Post(author_obj, content)
        self.posts.append(new_post)
        print(f"Post de '{author_name}' criado: '{content}'")
        return new_post

    def feed(self, user_name: str):
        if user_name not in self.users:
            print(f"ERRO FEED: Usuário '{user_name}' não encontrado.")
            return []

        user_obj = self.users[user_name]
        
        # Usuários que o user_name segue (out-neighbors no adj_list)
        following_users = set(self.adj_list.get(user_obj, []))
        
        # Incluir os próprios posts do usuário no feed
        feed_posts = [post for post in self.posts if post.author == user_obj]

        # Adicionar posts dos usuários que ele segue
        for post in self.posts:
            if post.author in following_users:
                feed_posts.append(post)
        
        # Ordenar posts por data (mais recentes primeiro)
        feed_posts.sort(key=lambda p: p.date, reverse=True)
        
        return feed_posts

    def display(self):
        print("\n--- Estado Atual da Rede UniPoli ---")
        print("\nUsuários:")
        if not self.users:
            print("Nenhum usuário cadastrado.")
        for user_name, user_obj in self.users.items():
            print(f"  - {user_name} ({user_obj.email})")

        print("\nConexões (Quem segue quem):")
        if not self.conections:
            print("Nenhuma conexão estabelecida.")
        for conn in self.conections:
            print(f"  - {conn.user_1.user} -> {conn.user_2.user} (Interações: {conn.n_interacoes})")
        
        print("\nLista de Adjacência:")
        if not self.adj_list:
             print("Lista de adjacência vazia.")
        for user_obj, neighbors in self.adj_list.items():
            neighbor_names = [n.user for n in neighbors]
            print(f"  - {user_obj.user}: {neighbor_names}")

        print("\nPosts:")
        if not self.posts:
            print("Nenhum post.")
        for post in self.posts:
            print(f"  - {post}")
        print("-------------------------------------")

    def updateLike(self, post_id: int):
        for post in self.posts:
            if post.post_id == post_id:
                post.likes += 1
                print(f"Post #{post_id} agora tem {post.likes} curtidas.")
                return True
        print(f"ERRO UPDATELIKE: Postagem com ID {post_id} não encontrada.")
        return False

    def bfs(self, start_user_name: str):
        print(f"\n--- Iniciando BFS a partir de: {start_user_name} ---")
        if start_user_name not in self.users:
            print(f"ERRO BFS: Usuário '{start_user_name}' não encontrado.")
            return None, None

        start_node = self.users[start_user_name]
        
        visited = {start_node}
        queue = deque([(start_node, 0)])  # (node, distance)
        traversal_order = []
        distances = {start_node: 0} # {User_obj: distance}

        while queue:
            current_node, dist = queue.popleft() # Usa popleft para BFS
            traversal_order.append(current_node.user) # Adiciona nome do usuário à ordem

            print(f"  Visitando: {current_node.user} (Distância: {dist})")

            # Agora usa a adj_list para encontrar vizinhos
            neighbors = self.adj_list.get(current_node, []) 
            
            for neighbor in neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    distances[neighbor] = dist + 1
                    queue.append((neighbor, dist + 1))
                    print(f"    Adicionado à fila: {neighbor.user}")

        # Formata distâncias para usar nomes de usuários
        formatted_distances = {user_obj.user: dist for user_obj, dist in distances.items()}
        print(f"--- BFS Concluída. Ordem: {traversal_order} ---")
        return traversal_order, formatted_distances

    def dfs(self, start_user_name: str):
        print(f"\n--- Iniciando DFS a partir de: {start_user_name} ---")
        if start_user_name not in self.users:
            print(f"ERRO DFS: Usuário '{start_user_name}' não encontrado.")
            return None

        start_node = self.users[start_user_name]
        
        visited = set()
        stack = [start_node]
        traversal_order = []

        while stack:
            current_node = stack.pop()
            if current_node not in visited:
                visited.add(current_node)
                traversal_order.append(current_node.user)
                print(f"  Visitando: {current_node.user}")

                # Obtém vizinhos da adj_list. Inverte a ordem para que os vizinhos com nomes menores
                # sejam processados primeiro se a ordem importa no stack (topo vs. fundo).
                # Se não ordenar, a ordem de visitação pode variar dependendo da inserção no dicionário.
                neighbors = sorted(self.adj_list.get(current_node, []), key=lambda u: u.user, reverse=True)
                
                for neighbor in neighbors:
                    if neighbor not in visited:
                        stack.append(neighbor)
                        print(f"    Adicionado à pilha: {neighbor.user}")
        
        print(f"--- DFS Concluída. Ordem: {traversal_order} ---")
        return traversal_order

    def recommend(self, user_ref_name: str):
        """
        Recomenda usuários para um determinado user_ref baseado em "amigos de amigos" (distância 2 via BFS),
        excluindo usuários já seguidos e o próprio usuário de referência.
        """
        print(f"\n--- Gerando recomendações para: {user_ref_name} ---")
        if user_ref_name not in self.users:
            print(f"ERRO RECOMEND: Usuário '{user_ref_name}' não encontrado.")
            return []

        # 1. Executar BFS para obter distâncias de todos os usuários a partir de user_ref
        _, distances_from_ref = self.bfs(user_ref_name)

        if distances_from_ref is None:
            print(f"Não foi possível calcular distâncias para recomendações de {user_ref_name}.")
            return []

        # 2. Identificar quem user_ref já segue (para não recomendar quem já é seguido)
        user_ref_obj = self.users[user_ref_name]
        
        # Usa adj_list para quem user_ref já segue
        already_following_objs = set(self.adj_list.get(user_ref_obj, []))
        already_following_names = {u_obj.user for u_obj in already_following_objs}

        recommended_users = []

        # 3. Filtrar usuários que estão a distância 2 e não são seguidos
        for user_name, distance in distances_from_ref.items():
            # Critérios para recomendação:
            # - Distância de 2
            # - Não é o próprio user_ref
            # - Não é alguém que user_ref já segue
            if distance == 2 and user_name != user_ref_name and user_name not in already_following_names:
                recommended_users.append(user_name)
        
        print(f"--- Recomendações para {user_ref_name}: {recommended_users} ---")
        return recommended_users

    # --- Novas Métricas da Ciência das Redes e Integração NetworkX ---

    def to_networkx_graph(self):
        """
        Converte a estrutura UniPoli para um objeto networkx.DiGraph.
        Isso é essencial para usar as funções avançadas do NetworkX.
        """
        G = nx.DiGraph() # Grafo direcionado para "segue"
        
        # Adiciona todos os nós (usuários)
        for user_name, user_obj in self.users.items():
            G.add_node(user_obj) # Adicionamos o objeto User como nó

        # Adiciona todas as arestas (conexões) com pesos
        for conn in self.conections:
            G.add_edge(conn.user_1, conn.user_2, weight=conn.n_interacoes)
        
        return G

    def get_network_metrics(self):
        """
        Calcula e retorna várias métricas da Ciência das Redes.
        """
        G = self.to_networkx_graph()

        metrics = {}

        # 1. Grau dos Nós (In-degree e Out-degree)
        metrics['degree'] = {node.user: G.degree(node) for node in G.nodes()} # Grau total (in+out)
        metrics['in_degree'] = {node.user: G.in_degree(node) for node in G.nodes()}
        metrics['out_degree'] = {node.user: G.out_degree(node) for node in G.nodes()}

        # 2. Centralidade
        # Degree Centrality (normalizada)
        if len(G) > 1: # Evita divisão por zero para grafos com 1 nó
            metrics['degree_centrality'] = {node.user: value for node, value in nx.degree_centrality(G).items()}
            metrics['in_degree_centrality'] = {node.user: value for node, value in nx.in_degree_centrality(G).items()}
            metrics['out_degree_centrality'] = {node.user: value for node, value in nx.out_degree_centrality(G).items()}
        else:
            metrics['degree_centrality'] = {node.user: 0 for node in G.nodes()}
            metrics['in_degree_centrality'] = {node.user: 0 for node in G.nodes()}
            metrics['out_degree_centrality'] = {node.user: 0 for node in G.nodes()}


        # Betweenness Centrality (pode ser computacionalmente cara para grafos muito grandes)
        # Calcula para o grafo direcionado
        metrics['betweenness_centrality'] = {node.user: value for node, value in nx.betweenness_centrality(G).items()}

        # Closeness Centrality (calcula para o maior componente fracamente conectado para grafos não-totalmente-conectados)
        metrics['closeness_centrality'] = {}
        # Converte para grafo não direcionado para closeness_centrality se necessário, ou lida com componentes
        # Para grafos direcionados, nx.closeness_centrality por padrão só considera caminhos acessíveis.
        # Pode ser undefined para nós inacessíveis.
        for component in nx.weakly_connected_components(G):
            subgraph = G.subgraph(component)
            if len(subgraph) > 1: # Closeness é 0 ou indefinida para nós isolados em componentes de 1 nó
                closeness = nx.closeness_centrality(subgraph)
                for node, value in closeness.items():
                    metrics['closeness_centrality'][node.user] = value
            else: # Caso de nó isolado
                for node in subgraph.nodes():
                    metrics['closeness_centrality'][node.user] = 0 # ou NaN, dependendo da interpretação
        

        # 3. Componentes Conectados (para grafos direcionados, geralmente "fracamente conectados")
        wccs = list(nx.weakly_connected_components(G))
        metrics['weakly_connected_components'] = [[node.user for node in component] for component in wccs]
        metrics['num_weakly_connected_components'] = len(wccs)
        
        # 4. Diâmetro da Rede
        # O diâmetro só é definido para grafos conectados. Para grafos direcionados, Strong ou Weakly.
        # Usaremos o maior componente fracamente conectado para calcular um "diâmetro aproximado" se o grafo não for um único WCC.
        metrics['diameter'] = "Não aplicável (grafo não conectado)"
        if metrics['num_weakly_connected_components'] == 1 and len(G) > 0:
            # Verifica se é um único componente conectado para calcular o diâmetro
            try:
                # Calcula o diâmetro do grafo (considerando-o não direcionado para esta métrica comum)
                # ou do maior componente fortemente conectado se for o caso de um grafo direcionado
                # Para simplificar para o projeto, usamos o grafo G (direcionado) e assumimos que se for 1 WCC, é válido.
                # nx.diameter exige que o grafo seja conectado.
                metrics['diameter'] = nx.diameter(nx.Graph(G)) # Converte para não-direcionado para o diâmetro padrão
            except nx.NetworkXError as e:
                metrics['diameter'] = f"Não aplicável (grafo não forte ou fracamente conectado o suficiente para diâmetro): {e}"
        elif len(G) > 0:
             # Para múltiplos componentes, podemos calcular o diâmetro do maior componente.
            if wccs:
                largest_wcc = max(wccs, key=len)
                if len(largest_wcc) > 1: # Diâmetro de componente de 1 nó é 0
                    subgraph_wcc = G.subgraph(largest_wcc)
                    try:
                        metrics['diameter'] = nx.diameter(nx.Graph(subgraph_wcc))
                    except nx.NetworkXError as e:
                        metrics['diameter'] = f"Diâmetro do maior componente não aplicável: {e}"
                else:
                    metrics['diameter'] = "Grafo possui apenas nós isolados ou componentes de tamanho 1."
            else:
                 metrics['diameter'] = "Grafo vazio ou sem componentes."
        else:
            metrics['diameter'] = "Grafo vazio"


        # 5. Densidade do Grafo
        metrics['density'] = nx.density(G)

        return metrics

    def visualize_graph(self, metrics_data=None):
        """
        Visualiza o grafo usando NetworkX e Matplotlib.
        Pode destacar nós com base nas métricas.
        """
        G = self.to_networkx_graph()

        if not G.nodes():
            messagebox.showinfo("Visualização do Grafo", "Grafo vazio. Adicione usuários e conexões primeiro.")
            return

        plt.figure(figsize=(12, 10))

        # Posições dos nós usando um layout de força (ex: spring_layout)
        # Isso ajuda a organizar os nós de forma visualmente agradável
        pos = nx.spring_layout(G, k=0.15, iterations=50) # k é a distância ideal entre nós

        node_colors = ['skyblue'] * len(G.nodes())
        node_sizes = [700] * len(G.nodes()) # Tamanho base dos nós
        edge_widths = [1] * len(G.edges())
        edge_alphas = [0.5] * len(G.edges())
        
        labels = {node: node.user for node in G.nodes()} # Usa o nome do usuário para o label

        if metrics_data and 'degree_centrality' in metrics_data:
            # Normaliza a centralidade para usar no tamanho dos nós
            degree_centrality_values = list(metrics_data['degree_centrality'].values())
            if degree_centrality_values: # Evita erro se a lista estiver vazia
                max_centrality = max(degree_centrality_values)
                min_centrality = min(degree_centrality_values)

                if max_centrality != min_centrality:
                    node_sizes = [
                        700 + 3000 * ((metrics_data['degree_centrality'][node.user] - min_centrality) / (max_centrality - min_centrality))
                        for node in G.nodes()
                    ]
                else: # Todos os nós têm a mesma centralidade
                     node_sizes = [700 + 1000] * len(G.nodes()) # Tamanho um pouco maior se todos forem iguais
            
            # Colore nós com maior in-degree ou out-degree (influência)
            # Exemplo: nós com out-degree alto (influenciadores que seguem muitos) ou in-degree alto (muito seguidos)
            if 'in_degree' in metrics_data and 'out_degree' in metrics_data:
                in_degrees = metrics_data['in_degree']
                out_degrees = metrics_data['out_degree']

                # Encontra o maior in-degree e out-degree para normalização ou destaque
                max_in_degree_val = max(in_degrees.values()) if in_degrees else 0
                max_out_degree_val = max(out_degrees.values()) if out_degrees else 0

                node_colors = []
                for node in G.nodes():
                    user_name = node.user
                    current_in_degree = in_degrees.get(user_name, 0)
                    current_out_degree = out_degrees.get(user_name, 0)

                    if max_in_degree_val > 0 and current_in_degree == max_in_degree_val:
                        node_colors.append('red') # NÓS MUITO SEGUIDOS (High In-degree)
                    elif max_out_degree_val > 0 and current_out_degree == max_out_degree_val:
                        node_colors.append('orange') # NÓS QUE SEGUEM MUITO (High Out-degree)
                    else:
                        node_colors.append('skyblue')
                else:
                    node_colors = ['skyblue'] * len(G.nodes())
            

        # Desenha os nós
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, alpha=0.9)
        # Desenha as arestas
        nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=edge_alphas, arrowstyle='->', arrowsize=20)
        # Desenha os labels dos nós
        nx.draw_networkx_labels(G, pos, labels, font_size=10, font_color='black')

        plt.title("Visualização da UniPoli", size=15)
        plt.axis('off') # Desliga os eixos
        plt.show()


In [107]:
## Teste
rede = UniPoli()

# Usuários
rede.addUser('Mikael', 'mikael@gmail.com')
rede.addUser('Madalena', 'Mad@gmail.com')
rede.addUser('Ana', 'Ana@gmail.com')
rede.addUser('Elon Musk', 'El@gmail.com')

rede.addUser('Maria', 'maria@gmail.com')
rede.addUser('Caio', 'caio@gmail.com')
rede.addUser('Otavio', 'ot@gmail.com')
rede.addUser('Julia', 'ju@gmail.com')

rede.addUser('Vitoria', 'vi@gmail.com')
rede.addUser('Bruno', 'bruno@gmail.com')
rede.addUser('Jose', 'jose@gmail.com')
rede.addUser('Trump', 'trump@gmail.com')

rede.addUser('Hanry', 'hanry@gmail.com')
rede.addUser('Chris', 'chris@gmail.com')
rede.addUser('Bob', 'bob@gmail.com')
rede.addUser('Robert', 'rb@gmail.com')

rede.addUser('Cleiton', 'cleiton@gmail.com')
rede.addUser('Nilton', 'nilton@gmail.com')
rede.addUser('Zé de manga', 'zdm@gmail.com')
rede.addUser('Zé do rádio', 'zdr@gmail.com')

# Conexões
rede.addConection('Mikael', 'Madalena', 1)
rede.addConection('Madalena', 'Mikael', 1)
rede.addConection('Madalena', 'Ana', 1)
rede.addConection('Ana', 'Madalena', 1)
rede.addConection('Mikael', 'Elon Musk', 100)

rede.display()


rede.delConection('Mikael', 'Elon Musk')
rede.delConection('Mikael', 'Ana')
rede.delUser('Elon Musk')
rede.create_post('Mikael', 'Eu sou muito esperto!')
rede.create_post('Ana', 'Mikael é um gostoso!')
rede.create_post('Madalena', 'Mikael é super inteligente!')

rede.updateLike(0)
rede.bfs('Ana')
rede.dfs('Ana')

rede.recommend('Mikael')

rede.display()

# Testar o feed
# Feed de Mikael
print("Feed de Mikael:")
for post in rede.feed('Mikael'):
    print(f"- {post}")

print('\n')
print("Feed de Madalena:")
for post in rede.feed('Madalena'):
    print(f"- {post}")

Usuário 'Mikael' adicionado com sucesso.
Usuário 'Madalena' adicionado com sucesso.
Usuário 'Ana' adicionado com sucesso.
Usuário 'Elon Musk' adicionado com sucesso.
Usuário 'Maria' adicionado com sucesso.
Usuário 'Caio' adicionado com sucesso.
Usuário 'Otavio' adicionado com sucesso.
Usuário 'Julia' adicionado com sucesso.
Usuário 'Vitoria' adicionado com sucesso.
Usuário 'Bruno' adicionado com sucesso.
Usuário 'Jose' adicionado com sucesso.
Usuário 'Trump' adicionado com sucesso.
Usuário 'Hanry' adicionado com sucesso.
Usuário 'Chris' adicionado com sucesso.
Usuário 'Bob' adicionado com sucesso.
Usuário 'Robert' adicionado com sucesso.
Usuário 'Cleiton' adicionado com sucesso.
Usuário 'Nilton' adicionado com sucesso.
Usuário 'Zé de manga' adicionado com sucesso.
Usuário 'Zé do rádio' adicionado com sucesso.
Conexão adicionada: 'Mikael' segue 'Madalena'.
Conexão adicionada: 'Madalena' segue 'Mikael'.
Conexão adicionada: 'Madalena' segue 'Ana'.
Conexão adicionada: 'Ana' segue 'Madalena

In [108]:
class Gui:
    def __init__(self, master):
        self.master = master
        master.title("UniPoli - Rede Social")
        master.geometry("400x300")
        master.resizable(width=False, height=False)

        self.rede = UniPoli()
        self.current_user = None

        self.create_login_screen()

        # Usuários
        self.rede.addUser('Mikael', 'mikael@gmail.com')
        self.rede.addUser('Madalena', 'Mad@gmail.com')
        self.rede.addUser('Ana', 'Ana@gmail.com')
        self.rede.addUser('Elon Musk', 'El@gmail.com')
        
        self.rede.addUser('Maria', 'maria@gmail.com')
        self.rede.addUser('Caio', 'caio@gmail.com')
        self.rede.addUser('Otavio', 'ot@gmail.com')
        self.rede.addUser('Julia', 'ju@gmail.com')
        
        self.rede.addUser('Vitoria', 'vi@gmail.com')
        self.rede.addUser('Bruno', 'bruno@gmail.com')
        self.rede.addUser('Jose', 'jose@gmail.com')
        self.rede.addUser('Trump', 'trump@gmail.com')
        
        self.rede.addUser('Hanry', 'hanry@gmail.com')
        self.rede.addUser('Chris', 'chris@gmail.com')
        self.rede.addUser('Bob', 'bob@gmail.com')
        self.rede.addUser('Robert', 'rb@gmail.com')
        
        self.rede.addUser('Cleiton', 'cleiton@gmail.com')
        self.rede.addUser('Nilton', 'nilton@gmail.com')
        self.rede.addUser('Zé de manga', 'zdm@gmail.com')
        self.rede.addUser('Zé do rádio', 'zdr@gmail.com')
        

    def clear_window(self):
        for widget in self.master.winfo_children():
            widget.destroy()

    def create_login_screen(self):
        self.clear_window()
        self.master.title("UniPoli - Login")

        tk.Label(self.master, text="Bem-vindo à UniPoli", font=('Times', 16, 'bold')).pack(pady=20)

        tk.Label(self.master, text="Nome de Usuário:").pack()
        self.username_entry = tk.Entry(self.master)
        self.username_entry.pack(pady=5)

        tk.Label(self.master, text="Email:").pack()
        self.email_entry = tk.Entry(self.master)
        self.email_entry.pack(pady=5)

        tk.Button(self.master, text="Login", command=self.login_user).pack(pady=10)
        tk.Button(self.master, text="Cadastrar", command=self.register_user).pack()

    def login_user(self):
        username = self.username_entry.get()
        email = self.email_entry.get()

        if username in self.rede.users and self.rede.users[username].email == email:
            self.current_user = self.rede.users[username]
            self.open_main_window()
        else:
            messagebox.showerror("Erro de Login", "Usuário ou email inválido.")

    def register_user(self):
        username = self.username_entry.get()
        email = self.email_entry.get()

        if not username or not email:
            messagebox.showwarning("Cadastro", "Por favor, preencha todos os campos.")
            return

        user = self.rede.addUser(username, email)
        if user:
            messagebox.showinfo("Cadastro", f"Usuário '{username}' cadastrado com sucesso!")
            self.create_login_screen() # Volta para tela de login
        else:
            messagebox.showerror("Cadastro", "Erro ao cadastrar usuário.")

    def open_main_window(self):
        self.clear_window()
        self.master.title(f"UniPoli - Logado como {self.current_user.user}")
        self.master.geometry('450x450')
        self.master.resizable(width=False, height=False)

        tk.Label(self.master, text=f"Bem-vindo, {self.current_user.user}!", font=('Times', 16, 'bold')).pack(pady=10)

        tk.Button(self.master, text="Feed de Postagens", command=self.open_feed_window).pack(pady=5)
        tk.Button(self.master, text="Criar Nova Postagem", command=self.open_create_post_window).pack(pady=5)
        tk.Button(self.master, text="Gerenciar Conexões", command=self.open_manage_connections_window).pack(pady=5)
        tk.Button(self.master, text="Algoritmos e Análise de Grafo", command=self.open_graph_algorithms_window).pack(pady=5)
        tk.Button(self.master, text="Logout", command=self.logout_user).pack(pady=20)
        tk.Button(self.master, text="Exibir Grafo (Console)", command=self.display_graph_in_console).pack()

    def logout_user(self):
        self.current_user = None
        messagebox.showinfo("Logout", "Saindo...")
        self.create_login_screen()

    def open_feed_window(self):
        feed_window = tk.Toplevel(self.master)
        feed_window.title("Meu Feed")
        feed_window.geometry("500x600")

        tk.Label(feed_window, text=f"Feed de {self.current_user.user}", font=('Times', 14, 'bold')).pack(pady=10)

        feed_frame = tk.Frame(feed_window)
        feed_frame.pack(expand=True, fill=tk.BOTH)

        feed_scroll = tk.Scrollbar(feed_frame)
        feed_scroll.pack(side=tk.RIGHT, fill=tk.Y)

        self.feed_text = tk.Text(feed_frame, wrap=tk.WORD, yscrollcommand=feed_scroll.set)
        self.feed_text.pack(expand=True, fill=tk.BOTH)
        feed_scroll.config(command=self.feed_text.yview)

        self.load_feed()

        # Campo para curtir
        like_frame = tk.Frame(feed_window)
        like_frame.pack(pady=10)
        tk.Label(like_frame, text="ID do Post para curtir:").pack(side=tk.LEFT)
        self.post_id_entry = tk.Entry(like_frame)
        self.post_id_entry.pack(side=tk.LEFT)
        tk.Button(like_frame, text="Curtir", command=self.like_post_action).pack(side=tk.LEFT, padx=5)

    def load_feed(self):
        self.feed_text.config(state=tk.NORMAL)
        self.feed_text.delete(1.0, tk.END)

        feed_posts = self.rede.feed(self.current_user.user)
        if not feed_posts:
            self.feed_text.insert(tk.END, "Nenhuma postagem no seu feed ainda.")
        else:
            for post in feed_posts:
                self.feed_text.insert(tk.END, f"Post #{post.post_id} - {post.author.user} ({post.date.strftime('%H:%M %d/%m')}):\n")
                self.feed_text.insert(tk.END, f"  \"{post.content}\"\n")
                self.feed_text.insert(tk.END, f"  Curtidas: {post.likes}\n\n")
        self.feed_text.config(state=tk.DISABLED)

    def like_post_action(self):
        post_id_str = self.post_id_entry.get()
        try:
            post_id = int(post_id_str)
            if self.rede.updateLike(post_id):
                self.load_feed() # Recarrega o feed para mostrar a curtida
            else:
                messagebox.showerror("Erro", "ID da postagem não encontrado.")
        except ValueError:
            messagebox.showerror("Erro", "Por favor, insira um ID de postagem válido (número inteiro).")


    def open_create_post_window(self):
        post_window = tk.Toplevel(self.master)
        post_window.title("Criar Nova Postagem")
        post_window.geometry("400x300")

        tk.Label(post_window, text=f"Quer postar o que hoje, {self.current_user.user}?", font=('Times', 12)).pack(pady=10)
        self.post_content_text = tk.Text(post_window, height=10, width=40)
        self.post_content_text.pack(pady=5)

        tk.Button(post_window, text="Postar", command=self.publish_post_action).pack(pady=10)

    def publish_post_action(self):
        content = self.post_content_text.get(1.0, tk.END).strip()
        if content:
            self.rede.create_post(self.current_user.user, content)
            self.post_content_text.delete(1.0, tk.END)
        else:
            messagebox.showwarning("Publicar Post", "A postagem não pode estar vazia.")

    def open_manage_connections_window(self):
        conn_window = tk.Toplevel(self.master)
        conn_window.title("Gerenciar Conexões")
        conn_window.geometry("500x400")

        tk.Label(conn_window, text="Gerenciar Conexões", font=('Times', 14, 'bold')).pack(pady=10)

        # Adicionar Conexão
        tk.Label(conn_window, text="Seguir Usuário:").pack()
        self.follow_entry = tk.Entry(conn_window)
        self.follow_entry.pack(pady=5)
        tk.Button(conn_window, text="Seguir", command=self.add_connection_action).pack(pady=5)

        # Remover Conexão
        tk.Label(conn_window, text="Deixar de Seguir Usuário:").pack()
        self.unfollow_entry = tk.Entry(conn_window)
        self.unfollow_entry.pack(pady=5)
        tk.Button(conn_window, text="Deixar de Seguir", command=self.del_connection_action).pack(pady=5)

        # Recomendações
        tk.Button(conn_window, text="Obter Recomendações", command=self.show_recommendations).pack(pady=10)

    def add_connection_action(self):
        user_to_follow = self.follow_entry.get()
        if user_to_follow and self.current_user:
            if user_to_follow == self.current_user.user:
                messagebox.showwarning("Erro", "Você não pode seguir a si mesmo.")
                return
            if self.rede.addConection(self.current_user.user, user_to_follow):
                messagebox.showinfo("Conexão", f"Você agora segue {user_to_follow}.")
            else:
                messagebox.showerror("Erro", "Não foi possível seguir o usuário.")
        else:
            messagebox.showwarning("Erro", "Por favor, digite um nome de usuário.")

    def del_connection_action(self):
        user_to_unfollow = self.unfollow_entry.get()
        if user_to_unfollow and self.current_user:
            if user_to_unfollow == self.current_user.user:
                messagebox.showwarning("Erro", "Você não pode deixar de seguir a si mesmo.")
                return
            if self.rede.delConection(self.current_user.user, user_to_unfollow):
                messagebox.showinfo("Conexão", f"Você deixou de seguir {user_to_unfollow}.")
            else:
                messagebox.showerror("Erro", "Não foi possível deixar de seguir o usuário.")
        else:
            messagebox.showwarning("Erro", "Por favor, digite um nome de usuário.")

    def show_recommendations(self):
        if not self.current_user:
            messagebox.showwarning("Recomendações", "Faça login para ver recomendações.")
            return

        recommendations = self.rede.recommend(self.current_user.user)
        if recommendations:
            messagebox.showinfo("Recomendações", "Usuários recomendados para você:\n" + ", ".join(recommendations))
        else:
            messagebox.showinfo("Recomendações", "Nenhuma recomendação disponível no momento.")


    def open_graph_algorithms_window(self):
        algo_window = tk.Toplevel(self.master)
        algo_window.title("Algoritmos de Grafo e Métricas")
        algo_window.geometry("600x400")

        tk.Label(algo_window, text="Algoritmos de Busca", font=('Times', 14, 'bold')).pack(pady=10)

        # BFS
        tk.Label(algo_window, text="Início BFS:").pack()
        self.bfs_start_entry = tk.Entry(algo_window)
        self.bfs_start_entry.pack(pady=2)
        tk.Button(algo_window, text="Executar BFS", command=self.run_bfs_action).pack(pady=5)

        # DFS
        tk.Label(algo_window, text="Início DFS:").pack()
        self.dfs_start_entry = tk.Entry(algo_window)
        self.dfs_start_entry.pack(pady=2)
        tk.Button(algo_window, text="Executar DFS", command=self.run_dfs_action).pack(pady=5)

        # --- Novas Seções ---
        tk.Label(algo_window, text="Análise da Rede", font=('Times', 14, 'bold')).pack(pady=10)

        # Botão para calcular e exibir métricas
        tk.Button(algo_window, text="Mostrar Métricas da Rede", command=self.show_network_metrics).pack(pady=5)

        # Botão para visualizar o grafo
        tk.Button(algo_window, text="Visualizar Rede", command=self.display_network_graph).pack(pady=5)

    def run_bfs_action(self):
        start_node_name = self.bfs_start_entry.get()
        if start_node_name:
            traversal_order, distances = self.rede.bfs(start_node_name)
            if traversal_order:
                messagebox.showinfo("BFS Resultado", 
                                    f"Ordem de Visita: {traversal_order}\n"
                                    f"Distâncias: {distances}")
            else:
                messagebox.showerror("Erro BFS", "Usuário inicial não encontrado ou erro na busca.")
        else:
            messagebox.showwarning("BFS", "Por favor, insira um usuário para iniciar a BFS.")

    def run_dfs_action(self):
        start_node_name = self.dfs_start_entry.get()
        if start_node_name:
            traversal_order = self.rede.dfs(start_node_name)
            if traversal_order:
                messagebox.showinfo("DFS Resultado", f"Ordem de Visita: {traversal_order}")
            else:
                messagebox.showerror("Erro DFS", "Usuário inicial não encontrado ou erro na busca.")
        else:
            messagebox.showwarning("DFS", "Por favor, insira um usuário para iniciar a DFS.")

    def show_network_metrics(self):
        """
        Calcula e exibe as métricas da rede em uma nova janela de texto.
        """
        if not self.rede.users:
            messagebox.showwarning("Métricas da Rede", "Adicione usuários para calcular as métricas.")
            return

        metrics = self.rede.get_network_metrics()

        metrics_window = tk.Toplevel(self.master)
        metrics_window.title("Métricas da Rede")
        metrics_window.geometry("800x600")

        metrics_text = tk.Text(metrics_window, wrap=tk.WORD)
        metrics_text.pack(expand=True, fill=tk.BOTH)
        metrics_text.config(state=tk.NORMAL) # Habilita para inserir texto

        metrics_text.insert(tk.END, "--- Métricas da Rede UniPoli ---\n\n")

        metrics_text.insert(tk.END, "1. Grau dos Nós:\n")
        metrics_text.insert(tk.END, "   - Grau Total (In + Out):\n")
        for user, degree in metrics['degree'].items():
            metrics_text.insert(tk.END, f"     {user}: {degree}\n")
        metrics_text.insert(tk.END, "   - Grau de Entrada (In-degree):\n")
        for user, degree in metrics['in_degree'].items():
            metrics_text.insert(tk.END, f"     {user}: {degree}\n")
        metrics_text.insert(tk.END, "   - Grau de Saída (Out-degree):\n")
        for user, degree in metrics['out_degree'].items():
            metrics_text.insert(tk.END, f"     {user}: {degree}\n")
        metrics_text.insert(tk.END, "\n")

        metrics_text.insert(tk.END, "2. Centralidade:\n")
        metrics_text.insert(tk.END, "   - Centralidade de Grau (Normalizada):\n")
        for user, centrality in metrics['degree_centrality'].items():
            metrics_text.insert(tk.END, f"     {user}: {centrality:.4f}\n")
        metrics_text.insert(tk.END, "   - Centralidade de Entrada (In-degree Centrality):\n")
        for user, centrality in metrics['in_degree_centrality'].items():
            metrics_text.insert(tk.END, f"     {user}: {centrality:.4f}\n")
        metrics_text.insert(tk.END, "   - Centralidade de Saída (Out-degree Centrality):\n")
        for user, centrality in metrics['out_degree_centrality'].items():
            metrics_text.insert(tk.END, f"     {user}: {centrality:.4f}\n")
        metrics_text.insert(tk.END, "   - Centralidade de Intermediação (Betweenness Centrality):\n")
        for user, centrality in metrics['betweenness_centrality'].items():
            metrics_text.insert(tk.END, f"     {user}: {centrality:.4f}\n")
        metrics_text.insert(tk.END, "   - Centralidade de Proximidade (Closeness Centrality):\n")
        for user, centrality in metrics['closeness_centrality'].items():
            metrics_text.insert(tk.END, f"     {user}: {centrality:.4f}\n")
        metrics_text.insert(tk.END, "\n")

        metrics_text.insert(tk.END, "3. Componentes Fracamente Conectados:\n")
        metrics_text.insert(tk.END, f"   - Número de Componentes: {metrics['num_weakly_connected_components']}\n")
        for i, component in enumerate(metrics['weakly_connected_components']):
            metrics_text.insert(tk.END, f"   - Componente {i+1}: {', '.join(component)}\n")
        metrics_text.insert(tk.END, "\n")

        metrics_text.insert(tk.END, "4. Diâmetro da Rede:\n")
        metrics_text.insert(tk.END, f"   - Diâmetro: {metrics['diameter']}\n")
        metrics_text.insert(tk.END, "\n")
        
        metrics_text.insert(tk.END, "5. Densidade do Grafo:\n")
        metrics_text.insert(tk.END, f"   - Densidade: {metrics['density']:.4f}\n")
        metrics_text.insert(tk.END, "\n")

        metrics_text.config(state=tk.DISABLED) # Desabilita para leitura

    def display_network_graph(self):
        """
        Chama o método de visualização do grafo da classe UniPoli, passando as métricas.
        """
        if not self.rede.users:
            messagebox.showwarning("Visualização do Grafo", "Adicione usuários para visualizar o grafo.")
            return
        
        # Opcional: calcular métricas e passá-las para a visualização para colorir/dimensionar
        metrics_data = self.rede.get_network_metrics()
        self.rede.visualize_graph(metrics_data)

    def display_graph_in_console(self):
        self.rede.display()
        messagebox.showinfo("Informação", "Detalhes do grafo foram impressos no console do Python.")



In [109]:
if __name__ == "__main__":
    root = tk.Tk()
    app = Gui(root)
    root.mainloop()

Usuário 'Mikael' adicionado com sucesso.
Usuário 'Madalena' adicionado com sucesso.
Usuário 'Ana' adicionado com sucesso.
Usuário 'Elon Musk' adicionado com sucesso.
Usuário 'Maria' adicionado com sucesso.
Usuário 'Caio' adicionado com sucesso.
Usuário 'Otavio' adicionado com sucesso.
Usuário 'Julia' adicionado com sucesso.
Usuário 'Vitoria' adicionado com sucesso.
Usuário 'Bruno' adicionado com sucesso.
Usuário 'Jose' adicionado com sucesso.
Usuário 'Trump' adicionado com sucesso.
Usuário 'Hanry' adicionado com sucesso.
Usuário 'Chris' adicionado com sucesso.
Usuário 'Bob' adicionado com sucesso.
Usuário 'Robert' adicionado com sucesso.
Usuário 'Cleiton' adicionado com sucesso.
Usuário 'Nilton' adicionado com sucesso.
Usuário 'Zé de manga' adicionado com sucesso.
Usuário 'Zé do rádio' adicionado com sucesso.


## Métricas
* 1. Complexidade temporal

In [101]:
# 1. Tempo de execução do bfs com vinte usuários na UniPoli
import time

start = time.time()
_ = rede.recommend('Mikael')
end = time.time()

print(f"Time taken: {end - start:.3f} seconds.")


--- Gerando recomendações para: Mikael ---

--- Iniciando BFS a partir de: Mikael ---
  Visitando: Mikael (Distância: 0)
    Adicionado à fila: Madalena
  Visitando: Madalena (Distância: 1)
    Adicionado à fila: Ana
  Visitando: Ana (Distância: 2)
--- BFS Concluída. Ordem: ['Mikael', 'Madalena', 'Ana'] ---
--- Recomendações para Mikael: ['Ana'] ---
Time taken: 0.000 seconds.
